In [2]:
import pandas as pd
import numpy
import xlrd
import re
import numpy as np
import matplotlib.pyplot as plt
import data

pd.set_option('display.max_columns', 500)

In [1]:
import sele_is_bad as sl
data = sl.ScrapMpg().data_for_match('https://api.monpetitgazon.com/championship/match/1060600')
data

Hello


stats,info_club,info_date_time,info_formation_place,info_formation_used,info_goals,info_idplayer,info_idteam,info_lastname,info_match_duration,info_match_id,...,stat_total_cross_nocorner,stat_total_offside,stat_total_scoring_att,stat_touches,stat_unsuccessful_touch,stat_was_fouled,stat_won_contest,stat_won_corners,stat_won_tackle,stat_yellow
id_player,,,,,,,,,,,,,,,,,,,,,
116504,Bordeaux,2019-09-28T15:30:00Z,4,3421,0,116504,140,Pablo,94,1060600,...,0,0,0,49,1,1,0,0,1,0
180835,Bordeaux,2019-09-28T15:30:00Z,7,3421,0,180835,140,Otávio,94,1060600,...,0,0,0,56,2,1,1,0,2,0
193512,Bordeaux,2019-09-28T15:30:00Z,0,3421,0,193512,140,Kamano,94,1060600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195480,Bordeaux,2019-09-28T15:30:00Z,0,3421,0,195480,140,Maja,94,1060600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201440,Bordeaux,2019-09-28T15:30:00Z,10,3421,0,201440,140,Hwang Ui-Jo,94,1060600,...,0,0,2,42,1,2,1,0,0,0
204715,Bordeaux,2019-09-28T15:30:00Z,0,3421,0,204715,140,Poussin,94,1060600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210228,Bordeaux,2019-09-28T15:30:00Z,0,3421,0,210228,140,Jovanovic,94,1060600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214472,Bordeaux,2019-09-28T15:30:00Z,0,3421,0,214472,140,Aït Bennasser,94,1060600,...,0,0,0,14,0,0,0,0,0,0
215046,Bordeaux,2019-09-28T15:30:00Z,2,3421,0,215046,140,Kwateng,94,1060600,...,0,0,0,73,0,2,3,0,2,0


In [102]:
import json
from urllib.request import urlopen
response = urlopen('https://api.monpetitgazon.com/championship/match/1060531')
y = json.loads(response.read())

In [103]:
def flatten_json(nested_json):
    """
        Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}
    
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

def transform_data(flat_json,team_type):
    df = pd.Series(flatten_json(flat_json[team_type]['players'])).to_frame()
    df['id_player'] = df.index
    df["id_player"] = df['id_player'].apply(lambda x : x.split('_')[0])
    df['stats'] = df.index
    df['stats'] = df['stats'].apply(lambda x : x.split('_')[1:])
    df['stats'] = df['stats'].apply(lambda x : '_'.join(x))
    df['value'] = df.iloc[:,0]
    
    df = df.reset_index()
    df = df.pivot(index = 'id_player', columns = 'stats', values = 'value')
    point_cols = [col for col in df2.columns if 'point' in col]
    df.drop(point_cols, axis = 1, inplace = True)
    
    df['info_where'] = team_type
    
    return df

    

home_team = transform_data(y,'Home')
away_team = transform_data(y,'Away')
match_data = pd.concat([home_team,away_team])
match_data


stats,info_formation_place,info_formation_used,info_goals,info_idplayer,info_idteam,info_lastname,info_mins_played,info_note_final_2015,info_own_goals,info_position,info_red_card,info_sub,info_yellow_card,stat_accurate_keeper_sweeper,stat_accurate_keeper_sweeper_point,stat_accurate_through_ball,stat_accurate_through_ball_point,stat_assist_attempt_saved,stat_assist_attempt_saved_point,stat_assist_blocked_shot,stat_assist_blocked_shot_point,stat_assist_free_kick_won_point,stat_assist_handball_won_point,stat_assist_own_goal,stat_assist_own_goal_point,stat_assist_pass_lost_point,stat_assist_penalty_won_point,stat_assist_post,stat_assist_post_point,stat_att_freekick_goal,stat_att_freekick_goal_point,stat_att_obox_goal,stat_att_obox_goal_point,stat_att_pen_goal,stat_att_pen_goal_point,stat_att_pen_miss,stat_att_pen_miss_point,stat_att_pen_post,stat_att_pen_post_point,stat_att_pen_target,stat_att_pen_target_point,stat_att_sv_high_centre,stat_att_sv_high_centre_point,stat_att_sv_high_left,stat_att_sv_high_left_point,stat_att_sv_high_right,stat_att_sv_high_right_point,stat_att_sv_low_centre,stat_att_sv_low_centre_point,stat_att_sv_low_left,stat_att_sv_low_left_point,stat_att_sv_low_right,stat_att_sv_low_right_point,stat_bad_clearance,stat_bad_clearance_point,stat_bad_contest,stat_bad_contest_point,stat_bad_keeper_sweeper,stat_bad_keeper_sweeper_point,stat_bad_tackle,stat_bad_tackle_point,stat_badcross,stat_badcross_point,stat_big_chance_created,stat_big_chance_created_point,stat_big_chance_missed,stat_big_chance_missed_point,stat_bonusRedCard,stat_bonusRedCard_point,stat_challenge_lost,stat_challenge_lost_point,stat_clean,stat_clean_point,stat_clearance_off_line,stat_clearance_off_line_point,stat_cross_not_claimed,stat_cross_not_claimed_point,stat_dangerous_play,stat_dangerous_play_point,stat_dispossessed,stat_dispossessed_point,stat_diving_save,stat_diving_save_point,stat_duel_lost,stat_duel_lost_point,stat_duel_won,stat_duel_won_point,stat_ecart,stat_ecart_point,stat_effective_blocked_cross,stat_effective_blocked_cross_point,stat_effective_clearance,stat_effective_clearance_point,stat_error_lead_to_goal,stat_error_lead_to_goal_point,stat_error_lead_to_shot,stat_error_lead_to_shot_point,stat_fouled_final_third,stat_fouled_final_third_point,stat_fouls,stat_fouls_point,stat_gk_smother,stat_gk_smother_point,stat_goal_assist,stat_goal_assist_intentional,stat_goal_assist_intentional_point,stat_goal_assist_point,stat_goals,stat_goals_conceded_without_pen,stat_goals_conceded_without_pen_point,stat_goals_point,stat_good_high_claim,stat_good_high_claim_point,stat_interception,stat_interception_point,stat_last_man_tackle,stat_last_man_tackle_point,stat_offside_provoked,stat_offside_provoked_point,stat_outfielder_block,stat_outfielder_block_point,stat_overrun,stat_overrun_point,stat_own_goals,stat_own_goals_point,stat_pass,stat_pass_point,stat_penalty_conceded,stat_penalty_conceded_point,stat_penalty_save,stat_penalty_save_point,stat_penalty_won,stat_penalty_won_point,stat_positionMPG,stat_poss_lost_ctrl,stat_poss_lost_ctrl_point,stat_poss_won_att_3rd,stat_poss_won_att_3rd_point,stat_poss_won_def_3rd,stat_poss_won_def_3rd_point,stat_poss_won_mid_3rd,stat_poss_won_mid_3rd_point,stat_post_scoring_att,stat_post_scoring_att_point,stat_punches,stat_punches_point,stat_put_through,stat_put_through_point,stat_red_card,stat_red_card_point,stat_resultat,stat_resultat_point,stat_saved_ibox,stat_saved_ibox_point,stat_saved_obox,stat_saved_obox_point,stat_score,stat_shot_off_target,stat_shot_off_target_point,stat_six_second_violation,stat_six_second_violation_point,stat_six_yard_block,stat_six_yard_block_point,stat_subReason,stat_subReason_point,stat_total_att_assist,stat_total_att_assist_point,stat_total_cross_nocorner,stat_total_cross_nocorner_point,stat_total_offside,stat_total_offside_point,stat_total_scoring_att,stat_total_scoring_att_point,stat_touches,stat_touches_point,stat_unsuccessful_touch,stat_unsuccessful_touch_point,stat_was_fouled,stat_was_fou

In [89]:
import pandas.io.json as pd_json

data = pd.read_json('https://api.monpetitgazon.com/championship/match/1060531')
data

,id,type,dateMatch,Home,Away,quotationPreGame,quotationPlayers,period,matchTime,definitiveRating
Away,match_1060531,match,2019-08-10T18:00:00Z,NaN,NaN,3.85,NaN,FullTime,96,True
Draw,match_1060531,match,2019-08-10T18:00:00Z,NaN,NaN,3.00,NaN,FullTime,96,True
Home,match_1060531,match,2019-08-10T18:00:00Z,NaN,NaN,2.25,NaN,FullTime,96,True
club,match_1060531,match,2019-08-10T18:00:00Z,Nice,Amiens,NaN,NaN,FullTime,96,True
id,match_1060531,match,2019-08-10T18:00:00Z,1395,1430,NaN,NaN,FullTime,96,True
player_104613,match_1060531,match,2019-08-10T18:00:00Z,NaN,NaN,NaN,3.80,FullTime,96,True
player_111181,match_1060531,match,2019-08-10T18:00:00Z,NaN,NaN,NaN,6.50,FullTime,96,True
player_111773,match_1060531,match,2019-08-10T18:00:00Z,NaN,NaN,NaN,16.00,FullTime,96,True
player_113541,match_1060531,match,2019-08-10T18:00:00Z,NaN,NaN,NaN,33.00,FullTime,96,True
player_116152,match_1060531,match,2019-08-10T18:00:00Z,NaN,NaN,NaN,5.25,FullTime,96,True


In [127]:
match_data['info_match_id'] = str(data['id']['id']).split('_')[1]
match_data['info_date_time'] = data['dateMatch']['id']


def get_team(x,data):
    if x == 'Home':
        team = str(data['Home']['club'])
    else : 
        team = str(data['Away']['club'])
    return team

def get_quote(x,data):
    try : 
        quote = data['quotationPlayers'][str('player_'+x)]
    except : 
        quote = None
        
    return quote

def get_win_team_quotation(x,data):
    try : 
        if x == 'Home':
            quote_win = data['quotationPreGame']['Home']
        else :
            quote_win = data['quotationPreGame']['Away']
    except : 
        quote_win = None
        quote_loose = None
    return quote_win

def get_loose_team_quotation(x,data):
    try : 
        if x == 'Away':
            quote_win = data['quotationPreGame']['Home']
        else :
            quote_win = data['quotationPreGame']['Away']
    except : 
        quote_win = None
        quote_loose = None
    return quote_win
    
    
match_data['info_club'] = match_data['info_where'].apply(lambda x:get_team(x,data))
match_data['info_quote_player'] = match_data['info_idplayer'].apply(lambda x:get_quote(x,data))
match_data['info_match_duration'] = data['matchTime']['id']

match_data['quote_win'] = match_data['info_where'].apply(lambda x:get_win_team_quotation(x,data))
match_data['quote_loose'] = match_data['info_where'].apply(lambda x:get_loose_team_quotation(x,data))
match_data['quote_draw'] = data['quotationPreGame']['Draw']

match_data = match_data.reindex(sorted(match_data.columns), axis=1)

In [128]:
match_data

stats,info_club,info_date_time,info_formation_place,info_formation_used,info_goals,info_idplayer,info_idteam,info_lastname,info_match_id,info_mins_played,info_note_final_2015,info_own_goals,info_position,info_quote_player,info_red_card,info_sub,info_where,info_yellow_card,quote_draw,quote_loose,quote_win,stat_accurate_keeper_sweeper,stat_accurate_keeper_sweeper_point,stat_accurate_through_ball,stat_accurate_through_ball_point,stat_assist_attempt_saved,stat_assist_attempt_saved_point,stat_assist_blocked_shot,stat_assist_blocked_shot_point,stat_assist_free_kick_won_point,stat_assist_handball_won_point,stat_assist_own_goal,stat_assist_own_goal_point,stat_assist_pass_lost_point,stat_assist_penalty_won_point,stat_assist_post,stat_assist_post_point,stat_att_freekick_goal,stat_att_freekick_goal_point,stat_att_obox_goal,stat_att_obox_goal_point,stat_att_pen_goal,stat_att_pen_goal_point,stat_att_pen_miss,stat_att_pen_miss_point,stat_att_pen_post,stat_att_pen_post_point,stat_att_pen_target,stat_att_pen_target_point,stat_att_sv_high_centre,stat_att_sv_high_centre_point,stat_att_sv_high_left,stat_att_sv_high_left_point,stat_att_sv_high_right,stat_att_sv_high_right_point,stat_att_sv_low_centre,stat_att_sv_low_centre_point,stat_att_sv_low_left,stat_att_sv_low_left_point,stat_att_sv_low_right,stat_att_sv_low_right_point,stat_bad_clearance,stat_bad_clearance_point,stat_bad_contest,stat_bad_contest_point,stat_bad_keeper_sweeper,stat_bad_keeper_sweeper_point,stat_bad_tackle,stat_bad_tackle_point,stat_badcross,stat_badcross_point,stat_big_chance_created,stat_big_chance_created_point,stat_big_chance_missed,stat_big_chance_missed_point,stat_bonusRedCard,stat_bonusRedCard_point,stat_challenge_lost,stat_challenge_lost_point,stat_clean,stat_clean_point,stat_clearance_off_line,stat_clearance_off_line_point,stat_cross_not_claimed,stat_cross_not_claimed_point,stat_dangerous_play,stat_dangerous_play_point,stat_dispossessed,stat_dispossessed_point,stat_diving_save,stat_diving_save_point,stat_duel_lost,stat_duel_lost_point,stat_duel_won,stat_duel_won_point,stat_ecart,stat_ecart_point,stat_effective_blocked_cross,stat_effective_blocked_cross_point,stat_effective_clearance,stat_effective_clearance_point,stat_error_lead_to_goal,stat_error_lead_to_goal_point,stat_error_lead_to_shot,stat_error_lead_to_shot_point,stat_fouled_final_third,stat_fouled_final_third_point,stat_fouls,stat_fouls_point,stat_gk_smother,stat_gk_smother_point,stat_goal_assist,stat_goal_assist_intentional,stat_goal_assist_intentional_point,stat_goal_assist_point,stat_goals,stat_goals_conceded_without_pen,stat_goals_conceded_without_pen_point,stat_goals_point,stat_good_high_claim,stat_good_high_claim_point,stat_interception,stat_interception_point,stat_last_man_tackle,stat_last_man_tackle_point,stat_offside_provoked,stat_offside_provoked_point,stat_outfielder_block,stat_outfielder_block_point,stat_overrun,stat_overrun_point,stat_own_goals,stat_own_goals_point,stat_pass,stat_pass_point,stat_penalty_conceded,stat_penalty_conceded_point,stat_penalty_save,stat_penalty_save_point,stat_penalty_won,stat_penalty_won_point,stat_positionMPG,stat_poss_lost_ctrl,stat_poss_lost_ctrl_point,stat_poss_won_att_3rd,stat_poss_won_att_3rd_point,stat_poss_won_def_3rd,stat_poss_won_def_3rd_point,stat_poss_won_mid_3rd,stat_poss_won_mid_3rd_point,stat_post_scoring_att,stat_post_scoring_att_point,stat_punches,stat_punches_point,stat_put_through,stat_put_through_point,stat_red_card,stat_red_card_point,stat_resultat,stat_resultat_point,stat_saved_ibox,stat_saved_ibox_point,stat_saved_obox,stat_saved_obox_point,stat_score,stat_shot_off_target,stat_shot_off_target_point,stat_six_second_violation,stat_six_second_violation_point,stat_six_yard_block,stat_six_yard_block_point,stat_subReason,stat_subReason_point,stat_total_att_assist,stat_total_att_assist_point,stat_total_cross_nocorner,stat_total_cross_nocorner_point,stat_total_offside,stat_total_offside_point,stat_total_scoring_att,stat_total_scoring_att_point,stat_touches,

In [129]:

m1 = pd.read_excel('matches0.xlsx')
m2 = pd.read_excel('matches1.xlsx')

In [131]:
print(m1.shape)
m1.head()

(36, 109)


,id_player,info_club,info_date_time,info_formation_place,info_formation_used,info_goals,info_idplayer,info_idteam,info_lastname,info_match_duration,info_match_id,info_mins_played,info_note_final_2015,info_own_goals,info_position,info_quote_player,info_red_card,info_sub,info_where,info_yellow_card,quote_draw,quote_loose,quote_win,stat_accurate_keeper_sweeper,stat_accurate_through_ball,stat_assist_attempt_saved,stat_assist_blocked_shot,stat_assist_own_goal,stat_assist_post,stat_att_freekick_goal,stat_att_obox_goal,stat_att_pen_goal,stat_att_pen_miss,stat_att_pen_post,stat_att_pen_target,stat_att_sv_high_centre,stat_att_sv_high_left,stat_att_sv_high_right,stat_att_sv_low_centre,stat_att_sv_low_left,stat_att_sv_low_right,stat_bad_clearance,stat_bad_contest,stat_bad_keeper_sweeper,stat_bad_tackle,stat_badcross,stat_big_chance_created,stat_big_chance_missed,stat_bonusRedCard,stat_challenge_lost,stat_clean,stat_clearance_off_line,stat_cross_not_claimed,stat_dangerous_play,stat_dispossessed,stat_diving_save,stat_duel_lost,stat_duel_won,stat_ecart,stat_effective_blocked_cross,stat_effective_clearance,stat_error_lead_to_goal,stat_error_lead_to_shot,stat_fouled_final_third,stat_fouls,stat_gk_smother,stat_goal_assist,stat_goal_assist_intentional,stat_goals,stat_goals_conceded_without_pen,stat_good_high_claim,stat_interception,stat_last_man_tackle,stat_offside_provoked,stat_outfielder_block,stat_overrun,stat_own_goals,stat_pass,stat_penalty_conceded,stat_penalty_save,stat_penalty_won,stat_positionMPG,stat_poss_lost_ctrl,stat_poss_won_att_3rd,stat_poss_won_def_3rd,stat_poss_won_mid_3rd,stat_post_scoring_att,stat_punches,stat_put_through,stat_red_card,stat_resultat,stat_saved_ibox,stat_saved_obox,stat_score,stat_shot_off_target,stat_six_second_violation,stat_six_yard_block,stat_subReason,stat_total_att_assist,stat_total_cross_nocorner,stat_total_offside,stat_total_scoring_att,stat_touches,stat_unsuccessful_touch,stat_was_fouled,stat_won_contest,stat_won_corners,stat_won_tackle,stat_yellow
0,113541,Nice,2019-08-10T18:00:00Z,5,433,1,113541,1395,Herelle,96,1060531,90,6.0,0,Defender,33.0,0,0,Home,0,3,3.85,2.25,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,8.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0,NaN,0.0,0.0,1.0,1.0,NaN,1.0,0.0,3.0,0.0,0.0,0.0,0.934783,0.0,NaN,0.0,D,6.0,0.0,5.0,2.0,0.0,NaN,1.0,0.0,V,NaN,NaN,137.0,0.0,NaN,0.0,0,1.0,0.0,0.0,1.0,99.0,0.0,0.0,0.0,1.0,1.0,0.0
1,115851,Nice,2019-08-10T18:00:00Z,0,433,0,115851,1395,Cardinale,96,1060531,0,NaN,0,Goalkeeper,NaN,0,1,Home,0,3,3.85,2.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,151873,Nice,2019-08-10T18:00:00Z,4,433,0,151873,1395,Cyprien,96,1060531,90,6.5,0,Midfielder,4.6,0,0,Home,0,3,3.85,2.25,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.878049,0.0,NaN,0.0,MD,14.0,0.0,1.0,6.0,0.0,NaN,3.0,0.0,V,NaN,NaN,174.0,4.0,NaN,0.0,0,5.0,1.0,0.0,4.0,102.0,1.0,3.0,1.0,1.0,0.0,0.0
3,152338,Nice,2019-08-10T18:00:00Z,7,433,0,152338,1395,Tameze,96,1060531,90,5.0,0,Midfielder,18.0,0,0,Home,1,3,3.85,2.25,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,NaN,0.0,0.0,NaN,4.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,0.0,NaN,0.0,MO,10.0,1.0,2.0,2.0,0.0,NaN,1.0,0.0,V,NaN,NaN,3.0,0.0,NaN,0.0,0,3.0,0.0,0.0,0.0,57.0,0.0,1.0,2.0,0.0,0.0,1.0
4,153649,Nice,2019-08-10T18:00:00Z,8,433,0,153649,1395,Walter,96,1060531,54,5.5,0,Midfielder,NaN,0,0,Home,0,3,3.85,2.25,Na

In [132]:
print(m2.shape)

(36, 109)


In [135]:
m1.columns

Index(['id_player', 'info_club', 'info_date_time', 'info_formation_place',
       'info_formation_used', 'info_goals', 'info_idplayer', 'info_idteam',
       'info_lastname', 'info_match_duration',
       ...
       'stat_total_cross_nocorner', 'stat_total_offside',
       'stat_total_scoring_att', 'stat_touches', 'stat_unsuccessful_touch',
       'stat_was_fouled', 'stat_won_contest', 'stat_won_corners',
       'stat_won_tackle', 'stat_yellow'],
      dtype='object', length=109)